In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

In [2]:
def text_cleaner(text):
    text = re.sub(r"--", " ", text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = " ".join(text.split())
    return text

In [3]:
persuasion = gutenberg.raw("austen-persuasion.txt")
alice = gutenberg.raw("carroll-alice.txt")

persuasion = re.sub(r"Chapter \d+", "", persuasion)
alice = re.sub(r"CHAPTER .*", "", alice)


In [4]:
alice = text_cleaner(alice)
persuasion = text_cleaner(persuasion)

In [5]:
nlp = spacy.load("en")
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [20]:
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

alice_sents_df = pd.DataFrame(alice_sents)
persuasion_sents_df = pd.DataFrame(persuasion_sents)

sentences = pd.DataFrame(alice_sents + persuasion_sents)

sentences.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [21]:
alice_sents_df.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [84]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]
    

In [85]:
# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 500 == 0:
            print("Processing row {}".format(i))
            
    return df

In [107]:
# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [108]:
word_counts_alice = bow_features(alice_sents_df[:200], common_words)

Processing row 0


In [109]:
word_counts_alice.tail()

,small,suitable,pennyworth,basil,observe,rapid,faith,blast,perception,seventeen,...,interested,velvet,earnestly,respectable,peg,illness,beau,worry,text_sentence,text_source
195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Would, YOU, like, cats, if, you, were, me, ?,...",Carroll
196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Well, ,, perhaps, not, ,, ', said, Alice, in,...",Carroll
197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(And, yet, I, wish, I, could, show, you, our, ...",Carroll
198,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(She, is, such, a, dear, quiet, thing, ,, ', A...",Carroll
199,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(and, she, is, such, a, nice, soft, thing, to,...",Carroll


In [110]:
word_counts_persuasion = bow_features(persuasion_sents_df[:200], common_words)

Processing row 0


In [111]:
word_counts_persuasion.head()

,small,suitable,pennyworth,basil,observe,rapid,faith,blast,perception,seventeen,...,interested,velvet,earnestly,respectable,peg,illness,beau,worry,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Sir, Walter, Elliot, ,, of, Kellynch, Hall, ,...",Austen
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(This, was, the, page, at, which, the, favouri...",Austen
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Walter, Elliot, ,, born, March, 1, ,, 1760, ,...",Austen
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(of, South, Park, ,, in, the, county, of, Glou...",Austen
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"("", Precisely, such, had, the, paragraph, orig...",Austen


In [112]:
word_counts = pd.concat([word_counts_alice, word_counts_persuasion])

In [114]:
word_counts.tail()

,small,suitable,pennyworth,basil,observe,rapid,faith,blast,perception,seventeen,...,interested,velvet,earnestly,respectable,peg,illness,beau,worry,text_sentence,text_source
195,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"("", Nay, ,, Sir, Walter, ,, "", cried, Mrs, Cla...",Austen
196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Have, a, little, mercy, on, the, poor, men, .)",Austen
197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(We, are, not, all, born, to, be, handsome, .)",Austen
198,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(The, sea, is, no, beautifier, ,, certainly, ;...",Austen
199,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(But, then, ,, is, not, it, the, same, with, m...",Austen


In [115]:
word_counts["text_source"].value_counts()

Austen     200
Carroll    200
Name: text_source, dtype: int64

In [116]:
z = {"Carroll": 1, "Austen": 0}

In [117]:
word_counts["text_source"] = word_counts["text_source"].map(z)

In [118]:
word_counts["text_source"].value_counts()

1    200
0    200
Name: text_source, dtype: int64

In [126]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier()
Y = word_counts["text_source"]
X = np.array(word_counts.loc[:, ~word_counts.columns.isin(["text_sentence", "text source"])])

#(word_counts.drop(["text_sentence", "text_source", 1]))

#x_train, y_train = df_train.loc[:, ~df_train.columns.isin(["successful"])], df_train["successful"]


In [127]:
Y.shape

(400,)

In [128]:
X.shape

(400, 3063)

In [129]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

In [130]:
rfc = rfc.fit(X_train, y_train)

print("Training set score:", rfc.score(X_train, y_train))
print("\nTest set score:", rfc.score(X_test, y_test))

Training set score: 1.0

Test set score: 1.0


In [131]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr = lr.fit(X_train, y_train)
print(X_train.shape, y_train.shape)
print("Training set score:", lr.score(X_train, y_train))
print("\nTest set score:", lr.score(X_test, y_test))

(240, 3063) (240,)
Training set score: 1.0

Test set score: 1.0


In [132]:
clf = ensemble.GradientBoostingClassifier()
clf = clf.fit(X_train, y_train)

print("Training set score:", clf.score(X_train, y_train))
print("\nTest set score:", clf.score(X_test, y_test))

Training set score: 1.0

Test set score: 1.0


In [133]:
emma = gutenberg.raw('austen-emma.txt')
emma = re.sub(r'VOLUME \w+', '', emma)
emma = re.sub(r'CHAPTER \w+', '', emma)
emma = text_cleaner(emma)
print(emma[:100])

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to


In [134]:
emma_doc = nlp(emma)

In [135]:
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]
emma_sents = [[sent, "Austen"] for sent in emma_doc.sents]

# Emma is quite long, let's cut it down to the same length as Alice.
emma_sents = emma_sents[0:len(alice_sents)]

In [137]:
emma_sentences = pd.DataFrame(emma_sents)
emma_bow = bow_features(emma_sentences[:200], common_words)

print('done')

Processing row 0
done


In [145]:
emma_bow.head()

,small,suitable,pennyworth,basil,observe,rapid,faith,blast,perception,seventeen,...,interested,velvet,earnestly,respectable,peg,illness,beau,worry,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Emma, Woodhouse, ,, handsome, ,, clever, ,, a...",Austen
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(She, was, the, youngest, of, the, two, daught...",Austen
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Her, mother, had, died, too, long, ago, for, ...",Austen
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Sixteen, years, had, Miss, Taylor, been, in, ...",Austen
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Between, _, them)",Austen


In [147]:
word_counts_emma_alice = pd.concat([emma_bow, word_counts_alice])

In [150]:
word_counts_emma_alice.head(5)

,small,suitable,pennyworth,basil,observe,rapid,faith,blast,perception,seventeen,...,interested,velvet,earnestly,respectable,peg,illness,beau,worry,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Emma, Woodhouse, ,, handsome, ,, clever, ,, a...",Austen
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(She, was, the, youngest, of, the, two, daught...",Austen
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Her, mother, had, died, too, long, ago, for, ...",Austen
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Sixteen, years, had, Miss, Taylor, been, in, ...",Austen
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Between, _, them)",Austen


In [157]:
m = {"Carroll": 1, "Austen": 0}

In [158]:
word_counts_emma_alice["text_source"] = word_counts_emma_alice["text_source"].map(m)

In [159]:
Y = word_counts_emma_alice["text_source"]
X = np.array(word_counts_emma_alice.loc[:, ~word_counts_emma_alice.columns.isin(["text_sentence", "text source"])])

In [160]:
X.shape

(400, 3063)

In [161]:
Y.shape

(400,)

In [162]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

In [163]:
lr = LogisticRegression()
lr = lr.fit(X_train, y_train)

print('\nTest set score:', lr.score(X_test, y_test))
lr_Emma_predicted = lr.predict(X_test)
pd.crosstab(y_test, lr_Emma_predicted)


Test set score: 1.0


col_0,0,1
text_source,,
0,83,0
1,0,77


### Challenge 0:
Recall that the logistic regression model's best performance on the test set was 93%. See what you can do to improve performance. Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires. Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.

In [176]:
import nltk

sentence = "So, she, was, considering, in, her, own, mind and"


text = nltk.Text(sentence)
len(text)

49

In [170]:
sentence = "So, she, was, considering, in, her, own, mind and"

len(sentence)

49

In [216]:
word_counts["sentence length"] = word_counts["text_sentence"].apply(len)


#for cell in word_counts["text_sentence"]:
 #   word_counts["sentence length"] = len(cell) 



#[word_counts["sentence length"] = len(sentence) for sentence in word_counts["text_sentence"]]
    


#word_counts["sentence length"] = len(str(word_counts["text_sentence"]))

In [253]:
word_counts.head()

,small,suitable,pennyworth,basil,observe,rapid,faith,blast,perception,seventeen,...,velvet,earnestly,respectable,peg,illness,beau,worry,text_sentence,text_source,sentence length
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",1,67
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",1,63
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",1,33
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(Oh, dear, !)",1,3
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",1,6


In [213]:
words = []
for token in sentence1:
    words.append(token)
            
    #return Counter(words)

In [214]:
words

[i, eat, ice, cream]

In [202]:
for token in nlp(sentence1):
    print(token.orth_, token.is_stop)

i True
eat False
ice False
cream False


In [339]:
def first_word(c):
    return c["text_sentence"][0]
    

In [340]:
word_counts["First_Word"] = word_counts.apply(first_word, axis=1)

In [354]:
word = "Alice"

word = nlp(word)
word[0].is_stop

False

In [367]:
def stop_word_first(c):
    word = c["First_Word"]
    #word = nlp(word)
    return word.is_stop

In [368]:
word_counts["Begins_With_Stop_Word"] = word_counts.apply(stop_word_first, axis=1)

In [369]:
word_counts.head(10)

,small,suitable,pennyworth,basil,observe,rapid,faith,blast,perception,seventeen,...,respectable,peg,illness,beau,worry,text_sentence,text_source,sentence length,First_Word,Begins_With_Stop_Word
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",1,67,Alice,False
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",1,63,So,False
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",1,33,There,False
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(Oh, dear, !)",1,3,Oh,False
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(I, shall, be, late, !, ')",1,6,I,False
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"((, when, she, thought, it, over, afterwards, ...",1,126,(,False
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(In, another, moment, down, went, Alice, after...",1,23,In,False
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(The, rabbit, -, hole, went, straight, on, lik...",1,44,The,False
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(Either, the, well, was, very, deep, ,, or, sh...",1,37,Either,False
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,"(First, ,, she, tried, to, look, down, and, ma...",1,49,First,False


In [268]:
word_counts.head(10)

,small,suitable,pennyworth,basil,observe,rapid,faith,blast,perception,seventeen,...,earnestly,respectable,peg,illness,beau,worry,text_sentence,text_source,sentence length,Begins_With_Stop_Word
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",1,67,No
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",1,63,No
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",1,33,No
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(Oh, dear, !)",1,3,No
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(I, shall, be, late, !, ')",1,6,No
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"((, when, she, thought, it, over, afterwards, ...",1,126,No
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(In, another, moment, down, went, Alice, after...",1,23,No
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(The, rabbit, -, hole, went, straight, on, lik...",1,44,No
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(Either, the, well, was, very, deep, ,, or, sh...",1,37,No
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,"(First, ,, she, tried, to, look, down, and, ma...",1,49,No


### Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work. This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

In [371]:
print(gutenberg.fileids())

['austen-emma.txt', 'austen-persuasion.txt', 'austen-sense.txt', 'bible-kjv.txt', 'blake-poems.txt', 'bryant-stories.txt', 'burgess-busterbrown.txt', 'carroll-alice.txt', 'chesterton-ball.txt', 'chesterton-brown.txt', 'chesterton-thursday.txt', 'edgeworth-parents.txt', 'melville-moby_dick.txt', 'milton-paradise.txt', 'shakespeare-caesar.txt', 'shakespeare-hamlet.txt', 'shakespeare-macbeth.txt', 'whitman-leaves.txt']


In [379]:
thurs = gutenberg.raw('chesterton-thursday.txt')
pattern = "[\[].*?[\]]"
thurs = re.sub(pattern, "", thurs)

#emma = re.sub(r'VOLUME \w+', '', emma)
#emma = re.sub(r'CHAPTER \w+', '', emma)
#emma = text_cleaner(emma)

thurs = text_cleaner(thurs)
print(thurs[:200])

To Edmund Clerihew Bentley A cloud was on the mind of men, and wailing went the weather, Yea, a sick cloud upon the soul when we were boys together. Science announced nonentity and art admired decay; 


In [380]:
thurs_doc = nlp(thurs)

In [381]:
thurs_sents = [[sent, "Chesterton"] for sent in thurs_doc.sents]

thurs_sents_df = pd.DataFrame(thurs_sents)

thurs_sents_df.head()

,0,1
0,"(To, Edmund, Clerihew, Bentley)",Chesterton
1,"(A, cloud, was, on, the, mind, of, men, ,, and...",Chesterton
2,"(Science, announced, nonentity, and, art, admi...",Chesterton
3,"(Like, the, white, lock, of, Whistler, ,, that...",Chesterton
4,"(Life, was, a, fly, that, faded, ,, and, death...",Chesterton


In [382]:
sentences_carroll_chesterton = pd.DataFrame(alice_sents + thurs_sents)

sentences_carroll_chesterton.head()

,0,1
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [383]:
sentences_carroll_chesterton.shape

(5175, 2)

In [388]:
alicewords = bag_of_words(alice_doc)
thurswords = bag_of_words(thurs_doc)

common_words = set(alicewords + thurswords)

In [389]:
word_counts_alice = bow_features(alice_sents_df[:200], common_words)

Processing row 0


In [390]:
word_counts_thurs = bow_features(thurs_sents_df[:200], common_words)

Processing row 0


In [391]:
word_counts_a_t = pd.concat([word_counts_alice, word_counts_thurs])

word_counts_a_t.head()

,small,combine,sufficiently,pennyworth,ale,observe,foremost,faith,darken,blast,...,leicester,velvet,earnestly,reveal,respectable,peg,beau,worry,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",Carroll
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",Carroll
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",Carroll


In [392]:
word_counts_a_t["text_source"].unique()

array(['Carroll', 'Chesterton'], dtype=object)

In [393]:
q = {"Carroll" : 1, "Chesterton": 0}

word_counts_a_t["text_source"] = word_counts_a_t["text_source"].map(q)

In [394]:
word_counts_a_t.head()

,small,combine,sufficiently,pennyworth,ale,observe,foremost,faith,darken,blast,...,leicester,velvet,earnestly,reveal,respectable,peg,beau,worry,text_sentence,text_source
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Alice, was, beginning, to, get, very, tired, ...",1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(So, she, was, considering, in, her, own, mind...",1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(There, was, nothing, so, VERY, remarkable, in...",1
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(Oh, dear, !)",1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,"(I, shall, be, late, !, ')",1


In [396]:
Y = word_counts_a_t["text_source"]
X = np.array(word_counts_a_t.loc[:, ~word_counts_a_t.columns.isin(["text_sentence", "text source"])])

In [397]:
X.shape

(400, 2983)

In [398]:
Y.shape

(400,)

In [399]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.4, random_state=0)

In [400]:
lr = LogisticRegression()
lr = lr.fit(X_train, y_train)

print('\nTest set score:', lr.score(X_test, y_test))
lr_predicted = lr.predict(X_test)
pd.crosstab(y_test, lr_predicted)


Test set score: 1.0


col_0,0,1
text_source,,
0,77,0
1,0,83
